# Pre-process phase 1

Based on Gru2Rec github repository by Balázs Hidasi  https://github.com/hidasib/GRU4Rec

Download data from http://2015.recsyschallenge.com/challenge.html



In [4]:
#load data
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime as dt
import os
from collections import  defaultdict 

PATH_TO_WORKING_DIR = 'c:/asi/dss/deepLearningRecsys/item_embedding_session_reco/'
PATH_TO_ORIGINAL_DATA = '././data/'
PATH_TO_PROCESSED_DATA = '././ppie_data/'

os.chdir(PATH_TO_WORKING_DIR)
data = pd.read_csv(PATH_TO_ORIGINAL_DATA + 'yoochoose_clicks_s.dat', sep=',', header=None, usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
data.columns = ['SessionId', 'TimeStr', 'ItemId']


In [5]:
#filter data
from collections import defaultdict
import re

# data['Time'] = data.TimeStr.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #This is not UTC. It does not really matter.
# del(data['TimeStr'])

data = data[data['TimeStr'].str.contains("2014-09")]    
data = data[data['TimeStr'].str.contains('....-..-1',regex=True)]    #filter only sessions from the 09-10


#remove sessions shorter than one click, and items who appear in less than 5 sessions
session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>1].index)]
item_supports = data.groupby('ItemId').size()
data = data[np.in1d(data.ItemId, item_supports[item_supports>=5].index)]
session_lengths = data.groupby('SessionId').size()
data = data[np.in1d(data.SessionId, session_lengths[session_lengths>=2].index)]
        

<p style="color:DarkMagenta;">**???** Why should we filter 1 click sessions? what is the percent of one click sessions </p>

In [4]:
#transfer to corpus format
sessions = data.groupby('SessionId')['ItemId'].apply(list)

with open(PATH_TO_PROCESSED_DATA + 'rcs15_sesss.txt',"w") as outfile:
    for session in sessions: 
        session_str = " ".join(str(x) for x in session)
        outfile.write(session_str+"\n")

<p style="color:DarkMagenta;">**???** Print the number of sessions and items in the dataset </p>

In [5]:
#split sessions to train and test
num_sessions = data.SessionId.nunique()
num_sessions_train = round(num_sessions*0.8)

with open (PATH_TO_PROCESSED_DATA + 'rcs15_sesss.txt',"r") as infile:
   content = infile.readlines()

with open (PATH_TO_PROCESSED_DATA + 'rcs15_sesss_tr.txt',"w") as outfile:
    for i in range(num_sessions_train):
        outfile.write(content[i].strip()+"\n")

with open (PATH_TO_PROCESSED_DATA + 'rcs15_sesss_ts.txt',"w") as outfile:
    for i in range(num_sessions_train,num_sessions):
            outfile.write(content[i].strip()+"\n")

